# Research Pipelines with Snakemake

**Reproducible Workflows for Energy System Analysis**

This notebook introduces Snakemake, a workflow management system that allows you to create reproducible and scalable data analyses. In energy research, we often need to process large datasets, run multiple scenarios, and generate reports - all in a reproducible manner.

## Learning Objectives

By the end of this notebook, you will:
1. Understand the need for workflow management in research
2. Learn the basics of Snakemake syntax and concepts
3. Create a complete energy system analysis pipeline
4. Understand how to parallelize and scale workflows
5. Learn best practices for reproducible research pipelines

## 1. Why Workflow Management?

Modern energy research involves complex data processing pipelines:

### Common Challenges:
- **Multiple Steps**: Data download → Processing → Modeling → Analysis → Visualization
- **Multiple Scenarios**: Different policy scenarios, years, regions
- **Long Computations**: Some models take hours or days to run
- **Reproducibility**: Need to re-run analysis with updated data or parameters
- **Collaboration**: Others need to reproduce your results
- **Scalability**: Running on different computing resources

### Traditional Approach Problems:
```bash
# Manual workflow - error-prone!
python download_data.py
python process_data.py scenario1
python process_data.py scenario2
python run_model.py scenario1
python run_model.py scenario2
python make_plots.py
python generate_report.py
```

**Problems:**
- No dependency tracking
- Manual parameter management  
- No parallelization
- Difficult to resume if something fails
- Hard to reproduce

## 2. Introduction to Snakemake

Snakemake is a workflow management system that:
- **Tracks dependencies** between files
- **Parallelizes** execution automatically
- **Resumes** from failures
- **Scales** from laptops to clusters
- **Ensures reproducibility**

### Key Concepts:

#### Rules
Define how to create output files from input files

#### Wildcards
Allow parameterization (like {scenario}, {year})

#### DAG (Directed Acyclic Graph)
Snakemake automatically builds a dependency graph

#### Configuration
External config files for parameters

In [ ]:
# Let's start by exploring the Snakemake workflow we've created
import os
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import yaml

# Check if we're in the right directory
os.chdir('../workflows')
print("Current directory:", os.getcwd())
print("Files in directory:", os.listdir('.'))

In [ ]:
# Let's examine the Snakefile structure
with open('Snakefile', 'r') as f:
    snakefile_content = f.read()

print("First 1000 characters of Snakefile:")
print(snakefile_content[:1000])

In [ ]:
# Let's examine the configuration file
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration structure:")
for key, value in config.items():
    print(f"{key}: {type(value).__name__}")
    if isinstance(value, dict) and len(value) < 10:
        for subkey in value.keys():
            print(f"  - {subkey}")
    elif isinstance(value, list) and len(value) < 10:
        print(f"  Values: {value}")

## 3. Understanding Snakemake Syntax

### Basic Rule Structure:

```python
rule rule_name:
    input:
        "path/to/input/file"
    output:
        "path/to/output/file"
    params:
        parameter="value"
    script:
        "scripts/script.py"
```

### Wildcards Example:

```python
rule process_scenario:
    input:
        "data/raw/input.csv"
    output:
        "results/processed_{scenario}_{year}.csv"
    params:
        scenario="{scenario}",
        year="{year}"
    script:
        "scripts/process.py"
```

This rule can create:
- `results/processed_baseline_2025.csv`
- `results/processed_high_renewable_2030.csv`
- etc.

## 4. Our Energy System Analysis Pipeline

Let's examine the workflow we've created step by step:

In [ ]:
# Let's visualize our workflow structure
try:
    # Generate the workflow graph
    result = subprocess.run(['snakemake', '--dag'], 
                          capture_output=True, text=True, cwd='.')
    
    if result.returncode == 0:
        print("Workflow DAG generated successfully!")
        print("\nFirst part of the DAG:")
        print(result.stdout[:500] + "...")
    else:
        print("Error generating DAG:")
        print(result.stderr)
        
except FileNotFoundError:
    print("Snakemake not installed. Let's examine the workflow structure manually.")
    
    # Parse the Snakefile to extract rules
    rules = []
    with open('Snakefile', 'r') as f:
        for line in f:
            if line.startswith('rule '):
                rule_name = line.split()[1].rstrip(':')
                rules.append(rule_name)
    
    print("Rules in our workflow:")
    for i, rule in enumerate(rules, 1):
        print(f"{i:2d}. {rule}")

## 5. Workflow Steps Explained

Our energy system analysis pipeline consists of several key steps:

### 1. Data Acquisition
- Download demand data, renewable profiles, network topology
- For this demo, we create realistic synthetic data

### 2. Data Processing  
- Clean and format data for each year
- Aggregate to appropriate temporal/spatial resolution

### 3. Network Building
- Create PyPSA networks for each scenario and year
- Apply scenario-specific parameters

### 4. Optimization
- Solve the linear programming problem
- Find optimal dispatch and capacity expansion

### 5. Visualization
- Generate plots for each scenario
- Maps, time series, generation mix

### 6. Analysis
- Calculate system metrics
- Compare scenarios
- Sensitivity analysis

### 7. Reporting
- Generate final HTML report
- Archive results

In [ ]:
# Let's examine one of our scripts to understand how Snakemake passes parameters
with open('scripts/download_data.py', 'r') as f:
    script_content = f.read()

print("Key parts of the download_data.py script:")
print("=" * 50)

# Extract the main function
lines = script_content.split('\n')
in_main = False
main_content = []

for line in lines:
    if 'def main():' in line:
        in_main = True
    elif in_main and line.startswith('def ') and 'main' not in line:
        break
    elif in_main:
        main_content.append(line)

print('\n'.join(main_content[:30]))  # Show first 30 lines of main()

## 6. Running the Workflow

### Basic Snakemake Commands:

```bash
# Show what would be run
snakemake --dry-run

# Run the workflow
snakemake

# Run with 4 parallel jobs
snakemake --cores 4

# Run specific target
snakemake results/summary_report.html

# Run specific scenario/year combination
snakemake results/plots/generation_mix_baseline_2025.png

# Show workflow graph
snakemake --dag | dot -Tpng > workflow.png

# Force re-run everything
snakemake --forceall

# Clean up output files
snakemake clean
```

In [ ]:
# Let's do a dry run to see what would be executed
try:
    result = subprocess.run(['snakemake', '--dry-run', '--quiet'], 
                          capture_output=True, text=True, cwd='.')
    
    if result.returncode == 0:
        print("Dry run successful! Here's what would be executed:")
        print(result.stdout)
    else:
        print("Dry run failed. This is expected if dependencies aren't installed.")
        print("Error:", result.stderr[:500])
        
except FileNotFoundError:
    print("Snakemake not installed. In a real environment, you would run:")
    print("conda install -c conda-forge snakemake")
    print("or")
    print("pip install snakemake")

## 7. Advanced Snakemake Features

### Configuration Files
Keep parameters separate from code:

```yaml
# config.yaml
scenarios:
  baseline:
    renewable_target: 0.3
  high_renewable:
    renewable_target: 0.8

years: [2025, 2030, 2035]
```

### Resource Management
Specify computational requirements:

```python
rule solve_network:
    input: "network.nc"
    output: "results.nc"
    resources:
        mem_mb=8000,    # 8GB RAM
        runtime=60      # 1 hour max
    script: "solve.py"
```

### Cluster Execution
Scale to high-performance computing:

```bash
# Submit to SLURM cluster
snakemake --cluster "sbatch -t {resources.runtime} --mem={resources.mem_mb}" \
          --jobs 100
```

## 8. Creating a Simple Example

Let's create a simplified workflow to demonstrate the concepts:

In [ ]:
# Create a simple example workflow
simple_snakefile = """
# Simple energy analysis workflow

SCENARIOS = ["low_renewable", "high_renewable"]

rule all:
    input:
        expand("simple_results/plot_{scenario}.png", scenario=SCENARIOS),
        "simple_results/comparison.csv"

rule generate_data:
    output:
        "simple_data/demand.csv",
        "simple_data/supply_{scenario}.csv"
    params:
        scenario="{scenario}"
    run:
        import pandas as pd
        import numpy as np
        import os
        
        os.makedirs("simple_data", exist_ok=True)
        
        # Generate demand data
        hours = range(24)
        demand = 1000 + 300 * np.sin(np.array(hours) * 2 * np.pi / 24)
        pd.DataFrame({"hour": hours, "demand": demand}).to_csv(output[0], index=False)
        
        # Generate supply data based on scenario
        if params.scenario == "low_renewable":
            renewable_share = 0.2
        else:
            renewable_share = 0.8
            
        renewable = renewable_share * demand * (0.5 + 0.5 * np.random.random(24))
        fossil = demand - renewable
        
        pd.DataFrame({
            "hour": hours, 
            "renewable": renewable, 
            "fossil": fossil
        }).to_csv(output[1], index=False)

rule create_plot:
    input:
        demand="simple_data/demand.csv",
        supply="simple_data/supply_{scenario}.csv"
    output:
        "simple_results/plot_{scenario}.png"
    params:
        scenario="{scenario}"
    run:
        import pandas as pd
        import matplotlib.pyplot as plt
        import os
        
        os.makedirs("simple_results", exist_ok=True)
        
        demand_df = pd.read_csv(input.demand)
        supply_df = pd.read_csv(input.supply)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        ax.fill_between(supply_df['hour'], 0, supply_df['fossil'], 
                       label='Fossil', color='gray', alpha=0.7)
        ax.fill_between(supply_df['hour'], supply_df['fossil'], 
                       supply_df['fossil'] + supply_df['renewable'], 
                       label='Renewable', color='green', alpha=0.7)
        ax.plot(demand_df['hour'], demand_df['demand'], 
               'k--', linewidth=2, label='Demand')
        
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Power (MW)')
        ax.set_title(f'Generation Mix - {params.scenario.replace("_", " ").title()}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(output[0], dpi=150, bbox_inches='tight')
        plt.close()

rule compare_scenarios:
    input:
        expand("simple_data/supply_{scenario}.csv", scenario=SCENARIOS)
    output:
        "simple_results/comparison.csv"
    run:
        import pandas as pd
        
        results = []
        for i, file in enumerate(input):
            df = pd.read_csv(file)
            scenario = SCENARIOS[i]
            
            total_renewable = df['renewable'].sum()
            total_fossil = df['fossil'].sum()
            renewable_share = total_renewable / (total_renewable + total_fossil)
            
            results.append({
                'scenario': scenario,
                'renewable_share': renewable_share,
                'total_renewable_mwh': total_renewable,
                'total_fossil_mwh': total_fossil
            })
        
        pd.DataFrame(results).to_csv(output[0], index=False)
"""

# Write the simple Snakefile
with open('Snakefile_simple', 'w') as f:
    f.write(simple_snakefile)

print("Created simple Snakefile example!")
print("This workflow will:")
print("1. Generate synthetic demand and supply data for two scenarios")
print("2. Create plots for each scenario")
print("3. Compare the scenarios in a summary table")

In [ ]:
# Let's try to run the simple workflow
try:
    # Clean up any existing files first
    import shutil
    for dir_name in ['simple_data', 'simple_results']:
        if os.path.exists(dir_name):
            shutil.rmtree(dir_name)
    
    # Run the simple workflow
    result = subprocess.run(['snakemake', '-s', 'Snakefile_simple', '--cores', '1'], 
                          capture_output=True, text=True, cwd='.')
    
    if result.returncode == 0:
        print("Simple workflow completed successfully!")
        print("\nOutput:")
        print(result.stdout)
        
        # Show generated files
        if os.path.exists('simple_results'):
            print("\nGenerated files:")
            for file in os.listdir('simple_results'):
                print(f"  - simple_results/{file}")
                
        # Show comparison results
        if os.path.exists('simple_results/comparison.csv'):
            comparison = pd.read_csv('simple_results/comparison.csv')
            print("\nScenario Comparison:")
            print(comparison.to_string(index=False))
    else:
        print("Workflow failed:")
        print(result.stderr)
        
except FileNotFoundError:
    print("Snakemake not available. Running the workflow manually...")
    
    # Execute the workflow steps manually
    import numpy as np
    
    # Create directories
    os.makedirs('simple_data', exist_ok=True)
    os.makedirs('simple_results', exist_ok=True)
    
    scenarios = ["low_renewable", "high_renewable"]
    
    # Generate data for each scenario
    hours = range(24)
    demand = 1000 + 300 * np.sin(np.array(hours) * 2 * np.pi / 24)
    pd.DataFrame({"hour": hours, "demand": demand}).to_csv('simple_data/demand.csv', index=False)
    
    results = []
    
    for scenario in scenarios:
        # Generate supply data
        renewable_share = 0.2 if scenario == "low_renewable" else 0.8
        renewable = renewable_share * demand * (0.5 + 0.5 * np.random.random(24))
        fossil = demand - renewable
        
        supply_df = pd.DataFrame({
            "hour": hours, 
            "renewable": renewable, 
            "fossil": fossil
        })
        supply_df.to_csv(f'simple_data/supply_{scenario}.csv', index=False)
        
        # Create plot
        fig, ax = plt.subplots(figsize=(10, 6))
        
        ax.fill_between(supply_df['hour'], 0, supply_df['fossil'], 
                       label='Fossil', color='gray', alpha=0.7)
        ax.fill_between(supply_df['hour'], supply_df['fossil'], 
                       supply_df['fossil'] + supply_df['renewable'], 
                       label='Renewable', color='green', alpha=0.7)
        ax.plot(hours, demand, 'k--', linewidth=2, label='Demand')
        
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Power (MW)')
        ax.set_title(f'Generation Mix - {scenario.replace("_", " ").title()}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'simple_results/plot_{scenario}.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # Calculate metrics
        total_renewable = supply_df['renewable'].sum()
        total_fossil = supply_df['fossil'].sum()
        renewable_share_actual = total_renewable / (total_renewable + total_fossil)
        
        results.append({
            'scenario': scenario,
            'renewable_share': renewable_share_actual,
            'total_renewable_mwh': total_renewable,
            'total_fossil_mwh': total_fossil
        })
    
    # Save comparison
    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv('simple_results/comparison.csv', index=False)
    
    print("Manual workflow execution completed!")
    print("\nScenario Comparison:")
    print(comparison_df.to_string(index=False))

## 9. Best Practices for Research Workflows

### 1. Modular Design
- Break complex analyses into small, focused steps
- Each rule should have a single, clear purpose
- Use meaningful names for rules and files

### 2. Configuration Management
```yaml
# config.yaml - Keep all parameters in one place
data:
  start_year: 2020
  end_year: 2050
  regions: ["north", "south", "east", "west"]

scenarios:
  baseline:
    co2_price: 0
    renewable_target: 0.3
  policy:
    co2_price: 100
    renewable_target: 0.8
```

### 3. Error Handling
- Validate inputs in your scripts
- Use informative error messages
- Log intermediate results for debugging

### 4. Documentation
```python
rule download_weather_data:
    """
    Download historical weather data from ECMWF ERA5 reanalysis.
    
    This rule fetches wind speed and solar irradiance data for the 
    specified years and regions. The data is used to calculate 
    renewable energy potential.
    """
    output:
        "data/raw/weather_{year}.nc"
    params:
        year="{year}"
    script:
        "scripts/download_weather.py"
```

### 5. Testing
- Create test data for development
- Test individual rules
- Use continuous integration

## 10. Integration with PyPSA

Our main workflow demonstrates how to integrate Snakemake with PyPSA for comprehensive energy system analysis:

### Key Integration Points:

#### 1. Data Preparation
```python
rule prepare_network_data:
    input:
        buses="data/raw/buses.csv",
        lines="data/raw/lines.csv",
        generators="data/raw/generators.csv"
    output:
        "data/processed/network_base.nc"
    script:
        "scripts/build_pypsa_network.py"
```

#### 2. Scenario Configuration
```python
rule apply_scenario:
    input:
        "data/processed/network_base.nc"
    output:
        "data/networks/network_{scenario}_{year}.nc"
    params:
        scenario_config=lambda w: config["scenarios"][w.scenario]
    script:
        "scripts/apply_scenario.py"
```

#### 3. Parallel Optimization
```python
rule solve_pypsa:
    input:
        "data/networks/network_{scenario}_{year}.nc"
    output:
        "results/solved/network_{scenario}_{year}.nc"
    resources:
        mem_mb=16000,  # 16GB for large networks
        runtime=120    # 2 hours max
    script:
        "scripts/solve_pypsa.py"
```

## 11. Scaling to High-Performance Computing

### Cluster Configuration

```yaml
# cluster.yaml - HPC job specifications
__default__:
  partition: "normal"
  time: "01:00:00"
  mem: "4G"
  nodes: 1
  ntasks: 1

solve_network:
  partition: "bigmem"
  time: "04:00:00"
  mem: "32G"
  ntasks: 8
```

### Submission Script
```bash
#!/bin/bash
# submit_workflow.sh

snakemake \
    --cluster-config cluster.yaml \
    --cluster "sbatch -p {cluster.partition} -t {cluster.time} --mem={cluster.mem}" \
    --jobs 50 \
    --latency-wait 60 \
    --rerun-incomplete
```

### Cloud Computing
```bash
# Run on Google Cloud with Kubernetes
snakemake --kubernetes \
    --default-remote-provider GS \
    --default-remote-prefix gs://my-energy-bucket
```

## 12. Reproducibility Features

### Software Environment
```yaml
# conda environment for each rule
rule solve_network:
    input: "network.nc"
    output: "results.nc"
    conda: "envs/pypsa.yaml"
    script: "solve.py"
```

### Container Support
```python
rule analyze_results:
    input: "results.nc"
    output: "analysis.html"
    container: "docker://pypsa/pypsa:latest"
    script: "analyze.py"
```

### Provenance Tracking
```bash
# Generate detailed report
snakemake --report report.html

# Archive workflow
snakemake --archive workflow_archive.tar.gz
```

## 13. Exercise: Build Your Own Workflow

Create a Snakemake workflow for a simple energy analysis:

### Requirements:
1. **Data Generation**: Create synthetic hourly load data for a week
2. **Processing**: Calculate daily and weekly statistics
3. **Visualization**: Create plots showing:
   - Hourly load profile
   - Daily load distribution
   - Weekly patterns
4. **Analysis**: Calculate peak demand, load factor, and variability metrics
5. **Report**: Generate a summary markdown report

### Workflow Structure:
```
exercise_workflow/
├── Snakefile
├── config.yaml
├── scripts/
│   ├── generate_load_data.py
│   ├── calculate_statistics.py
│   ├── create_plots.py
│   └── generate_report.py
└── results/
    ├── data/
    ├── plots/
    └── report.md
```

In [ ]:
# Exercise starter code
exercise_snakefile = """
# Exercise: Load Analysis Workflow

configfile: "exercise_config.yaml"

rule all:
    input:
        "exercise_results/report.md"

rule generate_load_data:
    output:
        "exercise_data/hourly_load.csv"
    params:
        config=config
    script:
        "exercise_scripts/generate_load_data.py"

# TODO: Add more rules for statistics, plots, and report
"""

exercise_config = """
# Exercise Configuration
simulation:
  days: 7
  base_load: 1000  # MW
  peak_factor: 1.5
  noise_level: 0.1

analysis:
  metrics:
    - "peak_demand"
    - "load_factor"
    - "variability"
"""

print("Exercise starter files:")
print("\n1. Snakefile:")
print(exercise_snakefile)
print("\n2. Config file:")
print(exercise_config)
print("\nYour task: Complete the workflow by adding the missing rules and scripts!")

## 14. Common Pitfalls and Solutions

### Problem 1: File Path Issues
```python
# Wrong - relative paths can be problematic
rule bad_example:
    script: "../scripts/process.py"

# Better - use consistent structure
rule good_example:
    script: "scripts/process.py"
```

### Problem 2: Hardcoded Parameters
```python
# Wrong - parameters in code
rule bad_parameters:
    shell: "python process.py --year 2025 --scenario baseline"

# Better - use config and params
rule good_parameters:
    params:
        year="{year}",
        scenario="{scenario}"
    shell: "python process.py --year {params.year} --scenario {params.scenario}"
```

### Problem 3: Missing Dependencies
```python
# Wrong - missing intermediate files
rule incomplete_deps:
    input: "raw_data.csv"
    output: "final_result.png"
    # Missing intermediate processing steps!

# Better - explicit dependency chain
rule process_data:
    input: "raw_data.csv"
    output: "processed_data.csv"

rule create_plot:
    input: "processed_data.csv"
    output: "final_result.png"
```

## 15. Summary and Next Steps

### What We've Learned:

1. **Workflow Management**: Essential for reproducible research
2. **Snakemake Basics**: Rules, wildcards, dependencies
3. **Configuration**: Separating parameters from code
4. **Parallelization**: Automatic scaling and resource management
5. **Integration**: How to combine with PyPSA and other tools
6. **Best Practices**: Modular design and documentation

### Benefits of Workflow Management:

- **Reproducibility**: Others can run your exact analysis
- **Efficiency**: Parallel execution and smart re-running
- **Scalability**: From laptop to supercomputer
- **Collaboration**: Clear structure for team projects
- **Documentation**: Workflow serves as methods documentation

### Next Steps:

1. **Practice**: Convert your existing analysis scripts to Snakemake
2. **Explore**: Try different executors (cluster, cloud)
3. **Integrate**: Combine with version control and continuous integration
4. **Contribute**: Share workflows with the research community
5. **Learn More**: Advanced features like checkpoints and sub-workflows

### Resources:

- **Snakemake Documentation**: [https://snakemake.readthedocs.io/](https://snakemake.readthedocs.io/)
- **Snakemake Tutorial**: [https://snakemake.readthedocs.io/en/stable/tutorial/tutorial.html](https://snakemake.readthedocs.io/en/stable/tutorial/tutorial.html)
- **Energy System Workflows**: Look for published research workflows on GitHub
- **Community**: Snakemake GitHub discussions and Stack Overflow

### Final Thoughts:

Workflow management is a crucial skill for modern energy research. As datasets grow larger and analyses become more complex, tools like Snakemake become indispensable for maintaining reproducible, scalable, and collaborative research practices.

Start small, build incrementally, and always think about the next person (including future you) who will need to understand and run your analysis!